In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the data
# folder from the pset folder
# e.g. 'cs231a/ps4'
FOLDERNAME = 'colmap'

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My\ Drive
%cd $FOLDERNAME

import json
import numpy as np

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/colmap


### Lego processing

In [5]:
def json_conversion(filename1, filename2, r, cax, output_file_name):

  # Intrinsic parameters
  """with open(filename1) as fh:
      for line in fh:
          command, description = line.strip().split(None, 1)
          if command.isnumeric():
            matrix = np.zeros((3, 3))
            params = description.split()
            f, cx, cy, k = float(params[3]), float(params[4]), float(params[5]), float(params[6])
            alpha, beta = f*k, f*k
            matrix[2][2] = 1
            matrix[0][0], matrix[1][1] = alpha, beta
            matrix[0][2], matrix[1][2] = cx, cy
            im_dict[str(command)] = [matrix]
            print("command: ", command)"""
  
  extrinsics = []
  file_names = []
  # Extrinsic parameters
  with open(filename2) as fh:
      for line in fh:
          command, description = line.strip().split(None, 1)
          params = description.split()
          if command.isnumeric():
            matrix = np.zeros((3, 4))
            
            # Quaternion to rotation matrix form
            qr, qi, qj, qk = float(params[0]), float(params[1]), float(params[2]), float(params[3])
            s = 1/(qr**2 + qi**2 + qj**2 + qk**2)
            
            tx, ty, tz = float(params[4]), float(params[5]), float(params[6])
            # rotation
            matrix[0][0] = 1 - 2 * s * (qj**2 + qk**2) 
            matrix[0][1] = 2 * s * (qi*qj - qk*qr)
            matrix[0][2] = 2 * s * (qi*qk + qj*qr)
      
            matrix[1][0] = 2 * s * (qi*qj + qk*qr)
            matrix[1][1] = 1 - 2 * s * (qi**2 + qk**2)
            matrix[1][2] = 2 * s * (qj*qk - qi*qr)
            
            matrix[2][0] = 2 * s * (qi * qk - qj * qr)
            matrix[2][1] = 2 * s * (qj * qk - qi * qr)
            matrix[2][2] = 1 - 2 * s * (qi**2 + qj**2) 

            # translation
            matrix[0][3], matrix[1][3], matrix[2][3] = tx, ty, tz

            extrinsics.append(matrix)
            t = str(params[8])
            file_names.append(params[8])

  M, names = [], []

  for extrinsic, name in zip(extrinsics, file_names):
    names.append(name)
    prod = extrinsic.tolist()
    prod.append([0.0, 0.0, 0.0, 1.0])
    M.append(np.array(prod))

  M = np.array(M)
  final_dict = {}
  dicts = []

  for num, array in zip(names, M):
    dictionary = {}
    dictionary["radius"] = r
    dictionary["transform_matrix"] = array.tolist()
    dictionary["file_path"] = str(num)
    dicts.append(dictionary)

  final_dict["camera_angle_x"] = cax
  final_dict["frames"] = dicts

  out_file = open(output_file_name, "w")
  json.dump(final_dict, out_file, indent = 4, sort_keys = False)
  out_file.close()

In [6]:
# Generate lego jsons
json_conversion('lego/lego_cameras_train.txt', 'lego/lego_images_train.txt', 6, 0.6911112070083618, "output/lego_new_test.json")
json_conversion('lego/lego_cameras_val.txt', 'lego/lego_images_val.txt', 6, 0.6911112070083618, "output/lego/lego_val.json")
json_conversion('lego/lego_cameras_test.txt', 'lego/lego_images_test.txt', 6, 0.6911112070083618, "output/lego/lego_test.json")

# Generate tank jsons
json_conversion('tank/train_cameras.txt', 'tank/images_train.txt', 4, 0.012062166059728024, "output/tank/tank_train.json")
json_conversion('tank/val_cameras.txt', 'tank/images_val.txt', 4, 0.012062166059728024, "output/tank/tank_val.json")
json_conversion('tank/test_cameras.txt', 'tank/images_test.txt', 4, 0.012062166059728024, "output/tank/tank_test.json")